[View in Colaboratory](https://colab.research.google.com/github/nenorden/Transfer-learning-on-Fashion-MNIST/blob/master/RandomForest.ipynb)

# Download Fashion-MNIST

In [1]:
import os
os.chdir('/content/')
!git clone https://github.com/zalandoresearch/fashion-mnist.git 
os.chdir('/content/fashion-mnist/')
import utils.mnist_reader as mnist_reader
os.chdir("/")
train_x_orig, train_y = mnist_reader.load_mnist('content/fashion-mnist/data/fashion', kind='train')
test_x_orig, test_y = mnist_reader.load_mnist('content/fashion-mnist/data/fashion', kind='t10k')


Cloning into 'fashion-mnist'...
remote: Counting objects: 615, done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 615 (delta 5), reused 9 (delta 4), pack-reused 603
Receiving objects: 100% (615/615), 105.18 MiB | 56.24 MiB/s, done.
Resolving deltas: 100% (347/347), done.


# Import ResNet Features

## Download data from Drive to virtual machine

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Eliza's file id's
#train_file_id = '1j1488T_da1oVlwwy-_19GMuY9tSucqzg'
#test_file_id = '1qR7fOsQ6TTW8eV8SnKN-jVvswlzMx9QM'

# Alex' file id's
train_file_id = '1u9MTHtqwDtHL3z6NHMP8xRJw_54Fag5T'
test_file_id = '1kkbjDm6hilfA4PFcxBgRGrN8gcGY6dmn'

os.chdir('/content/')
# Load files by ID
downloaded = drive.CreateFile({'id': train_file_id})
downloaded.GetContentFile('train_fashion-MNIST_resnet.npy')

downloaded = drive.CreateFile({'id': test_file_id})
downloaded.GetContentFile('test_fashion-MNIST_resnet.npy')

# Files should be in /content/. Check!

!ls
os.chdir('/')


datalab        test_fashion-MNIST_resnet.npy
fashion-mnist  train_fashion-MNIST_resnet.npy


## Load ResNet data

In [0]:
%matplotlib inline
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt

os.chdir('/content/')
train_x = np.load('train_fashion-MNIST_resnet.npy')
test_x = np.load('test_fashion-MNIST_resnet.npy')
os.chdir('/')


# Random Forest

## ResNet features

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

# Hyperparameters
param_grid = [{'max_depth': [None, 10], 'n_estimators': [100], 
               'max_features': ['sqrt', None]}]

# Create model and fit to training data.
# Perform grid search CV to find the best hyperparameters.
start_time = time.time()
rf = RandomForestClassifier(n_jobs=-1)
rf = GridSearchCV(rf, param_grid)
rf.fit(train_x, train_y)

print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(rf)

# Predict on test data
rf_predict = rf.predict(test_x)

# Get accuracy
rf_acc = (rf_predict == test_y).mean()
print(rf_acc)

--- 10637.147904396057 seconds ---
GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [None, 10], 'n_estimators': [100], 'max_features': ['sqrt', None]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
0.6263


## Original features

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

# Hyperparameters
param_grid = [{'max_depth': [None, 10], 'n_estimators': [100],
               'max_features': ['sqrt', None]}]

# Create model and fit to training data.
# Perform grid search CV to find the best hyperparameters.
start_time = time.time()

rf_orig = RandomForestClassifier(n_jobs=-1)
rf_orig = GridSearchCV(rf_orig, param_grid)
rf_orig.fit(train_x_orig, train_y)

print("--- %s seconds ---" % (time.time() - start_time))

# print model with chosen parameters
print(rf_orig)

# Predict on test data
rf_predict_orig = rf_orig.predict(test_x_orig)

# Get accuracy
rf_acc_orig = (rf_predict_orig == test_y).mean()
print(rf_acc_orig)
# 0.8774

--- 6703.958931207657 seconds ---
GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [None, 10], 'n_estimators': [100], 'max_features': ['sqrt', None]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
0.8763
